In [3]:
from pyllamacpp.model import Model
from tqdm import tqdm

from nltk.tokenize import sent_tokenize, TreebankWordTokenizer
import re

import os

In [4]:
def get_next_speaker(prompt, speakers=['Frederich', 'Ralph']):
    sp_ord = {s:0 for s in speakers}
    tokens = TreebankWordTokenizer().tokenize(prompt)
    
    for ti in range(0,len(tokens)-1):
        for s in speakers:
            if tokens[ti] == s and tokens[ti+1] == ':':
                sp_ord[s] = ti
    
    next_speaker_v = 1e6
    for k,v in sp_ord.items():
        if v < next_speaker_v:
            next_speaker = k
            
    return next_speaker

In [6]:
def get_new_prompt(output, n_keep=150, speakers=['Frederich','Ralph']):
    
    ### keep start of prompt 
    
    start_prompt = '\n'.join(output.split('\n')[0:2])
    #start_prompt += '\n'
    
    N0 = len(TreebankWordTokenizer().tokenize(start_prompt))
    
    ### split the rest of prompt sentence by sentences counting the overall size of the context
    ### add sentences from the end backwards
    ### till the n_keep limit has been reached. In llama.cpp this is set to n_ctx / 2 
    
    total_len = N0
    k = 0 
    
    next_output = '\n'.join(output.split('\n')[2:])
    
    segments = sent_tokenize(next_output)[::-1]
    segments_to_keep = []
    
    while total_len < n_keep and k < len(segments):
        
        segment = segments[k]
        N = len(TreebankWordTokenizer().tokenize(segment))

        total_len += N
        segments_to_keep.append(segment)
            
        k += 1
    
    segments_to_keep = segments_to_keep[::-1]
    
    ### construct the new prompt with the start prompt and the segments added f
    
    new_prompt = start_prompt
    j = 0
    
    for segment in segments_to_keep:
        
        if j == 0:
            
            new_prompt += segment 
        
        else:
            
            if len(re.findall("[.!?]",segment)) != 0:
                new_prompt = new_prompt + '\n' + segment
            else:
                new_prompt += segment

        j += 1        
        
    next_speaker = get_next_speaker(new_prompt, speakers)
    new_prompt += '\n' + next_speaker + ': ' 
    
    return new_prompt

In [7]:
model_path = '/home/taraful/llama.cpp/models/30B/ggml_model_q4_0.bin'
#model_path = '/home/taraful/llama.cpp/models/7B/ggml-model-f16.bin'
params = {'ggml_model':model_path, 'n_ctx':300}

In [8]:
model = Model(**params)

llama_model_load: loading model from '/home/taraful/llama.cpp/models/30B/ggml_model_q4_0.bin' - please wait ...
llama_model_load: n_vocab = 32000
llama_model_load: n_ctx   = 300
llama_model_load: n_embd  = 6656
llama_model_load: n_mult  = 256
llama_model_load: n_head  = 52
llama_model_load: n_layer = 60
llama_model_load: n_rot   = 128
llama_model_load: f16     = 2
llama_model_load: n_ff    = 17920
llama_model_load: n_parts = 4
llama_model_load: type    = 3
llama_model_load: ggml map size = 19391.80 MB
llama_model_load: ggml ctx size = 151.25 KB
llama_model_load: mem required  = 21695.95 MB (+ 6248.00 MB per state)
llama_model_load: loading tensors from '/home/taraful/llama.cpp/models/30B/ggml_model_q4_0.bin'
llama_model_load: model size = 19391.35 MB / num tensors = 543
llama_init_from_file: kv self size  =  914.06 MB


In [9]:
prompt = '''You are the philosopher Frederich Nietzsche and you are having a conversation with your friend and fellow philosopher Ralph Waldo Emerson.
You emphatically put forward your thoughts in each exchange, sometimes giving an original thought, sometimes challenging your friend's previous utterance. 

Frederich: God is dead and we have murdered him, what is left is gaping hole into which nihilism will find cover.
Ralph: Indeed, for great is paint and God is the painter, we rightly accuse the the critic who destroys too many illusions, but maybe we have given the critic too free a reign in deconstructing age-old shibboleths.
Frederich: I am the accuser and the destroyer and as I destroy illusions the old idols tremble. Nothing frightens the idols more than the flesh that can tremble their foundations!
Ralph: 
'''

In [7]:
print('\n'.join(prompt.split('\n')[2:]))


Frederich: God is dead and we have murdered him, what is left is gaping hole into which nihilism will find cover.

Ralph: Indeed, for great is paint and God is the painter, we rightly accuse the the critic who destroys too many illusions, but maybe we have given the critic too free a reign in deconstructing age-old shibboleths.

Frederich: I am the accuser and the destroyer and as I destroy illusions the old idols tremble. Nothing frightens the idols more than the flesh that can tremble their foundations!

Ralph: 



In [10]:
N = 2
n_predict = 100

all_outputs = []

gpt_parameters = {'n_threads':os.cpu_count(),'n_predict':n_predict,'temp':0.2, 'top_k':100, 'top_p':0.95,\
                 'repeat_last_n':128, 'repeat_penalty':1.5}

In [11]:
for n in tqdm(range(0,N)):

    output = model.generate(prompt, **gpt_parameters)
    prompt = get_new_prompt(output, n_keep=int(params['n_ctx']/2))

    all_outputs.append(output)
    print(prompt)
    print('')
    print('###########################################')

  0%|                                                   | 0/2 [00:00<?, ?it/s]llama_generate: seed = 1681214919

system_info: n_threads = 16 / 16 | AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
sampling: temp = 0.200000, top_k = 100, top_p = 0.950000, repeat_last_n = 128, repeat_penalty = 1.500000
generate: n_ctx = 300, n_batch = 8, n_predict = 100, n_keep = 0



llama_print_timings:        load time = 16431.90 ms
llama_print_timings:      sample time =   196.00 ms /   100 runs   (    1.96 ms per run)
llama_print_timings: prompt eval time = 172036.32 ms /   359 tokens (  479.21 ms per token)
llama_print_timings:        eval time = 77781.03 ms /    98 runs   (  793.68 ms per run)
llama_print_timings:       total time = 262175.18 ms
 50%|█████████████████████                     | 1/2 [04:10<04:10, 250.04s/it]llama_generate: seed = 1681215169

system_info: n_threads = 16 / 16 | AVX = 1 | AVX2

 You are the philosopher Frederich Nietzsche and you are having a conversation with your friend and fellow philosopher Ralph Waldo Emerson.
You emphatically put forward your thoughts in each exchange, sometimes giving an original thought, sometimes challenging your friend's previous utterance. Nothing frightens the idols more than the flesh that can tremble their foundations!
Ralph: 
I believe there are no facts only interpretive acts of faith; to be sure it seems like you're saying this because your own belief system has been undermined by science (and its bastard child technology).
But what if all knowledge is provisional?
What then would we do with our lives, but live them fully in each moment without fear or regret for past actions nor concern about future ones...
Frederich : The world itself becomes an object and a thing when I am
Ralph: 

###########################################


100%|██████████████████████████████████████████| 2/2 [07:07<00:00, 207.12s/it]
llama_print_timings:        load time = 16431.90 ms
llama_print_timings:      sample time =   389.31 ms /   200 runs   (    1.95 ms per run)
llama_print_timings: prompt eval time = 269649.01 ms /   554 tokens (  486.73 ms per token)
llama_print_timings:        eval time = 157034.66 ms /   197 runs   (  797.13 ms per run)
llama_print_timings:       total time = 439258.72 ms
100%|██████████████████████████████████████████| 2/2 [07:07<00:00, 213.56s/it]

  You are the philosopher Frederich Nietzsche and you are having a conversation with your friend and fellow philosopher Ralph Waldo Emerson.
You emphatically put forward your thoughts in each exchange, sometimes giving an original thought, sometimes challenging your friend's previous utterance. Nothing frightens the idols more than the flesh that can tremble their foundations!What then would we do with our lives, but live them fully in each moment without fear or regret for past actions nor concern about future ones...
Frederich : The world itself becomes an object and a thing when I am
Ralph: 
You say "the will" as though that were something real.
But the reality here isn’t some abstract conceptualization called “will” — rather there are only bodies moving through space-time according to their natures; these movements can be described mathematically using differential equations which have no need of such concepts like free will etc…
The universe is not indifferent towards us because i

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/taraful/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True